In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tqdm import tqdm
import os
import shutil

!pip install tensorboardcolab
import tensorboardcolab

import sys

Using TensorFlow backend.


In [0]:
# for i in range(100000):
#     sys.stdout.write('/r progress : {}\n'.format(i))
#     sys.stdout.flush()
#     if i == 1000 :
#         print('/n hi')

In [0]:
from tensorflow.keras.datasets.cifar10 import load_data
(train_x, train_y), (test_x, test_y) = load_data()

In [0]:
test_y = test_y.reshape(-1)

In [0]:
class Dataset:
    # 데이터셋을 배치 단위로 처리할 수 있도록 도와주는 Class
    def __init__(self, images, labels):
        self.images = images.copy()
        self.labels = labels.copy()
        self.labels = self.labels.ravel() # (None,1) -> (None,)
        self.counter = 0
        
    def __len__(self):
        # 데이터의 갯수를 반환
        # e.g) len(dataset)
        return len(self.images)

    def next_batch(self, batch_size=32):
        if self.counter + batch_size > len(self.images):
            # counter가 데이터셋의 갯수보다 초과한 경우,
            # 데이터셋을 섞어줌
            self.shuffle()
            self.counter = 0
        batch_images = self.images[self.counter:self.counter+batch_size]
        batch_labels = self.labels[self.counter:self.counter+batch_size]
        self.counter+=batch_size
        return batch_images.copy(), batch_labels.copy()
    
    def shuffle(self):
        # 데이터 셋을 섞어주는 함수
        indices = np.arange(len(self.images))
        np.random.shuffle(indices)        
        self.images = self.images[indices]
        self.labels = self.labels[indices]

In [0]:
def batch_norm_cnn (inputs, is_train) :
    # Eps, decay
    epsilon = 1e-5
    decay = 0.9999
    
    #input shape의 feature size측정
    sizes = inputs.get_shape()[-1]
    
    #test를 위한 moving average를 계산위함, 초기값은 0으로 지정 : tf.zeros
    test_mean = tf.Variable(tf.zeros([sizes]), name = 'test_mean')
    test_var = tf.Variable(tf.zeros([sizes]), name = 'test_var')
    
    # featuare갯수만큼 scale factor와 shiftf factor
    gamma = tf.Variable(tf.ones(sizes), name='scale_factor')
    beta = tf.Variable(tf.zeros(sizes), name='shift_factor')

    def train():
        mean, var = tf.nn.moments(inputs, [0, 1, 2])
        updated_mean = tf.assign(test_mean, 
                                 (((1-decay) * mean) + (decay*(test_mean))))
        updated_var = tf.assign(test_var,
                                (((1-decay) * var) + (decay*(test_var))))
        
        with tf.control_dependencies([updated_mean ,updated_var]) :
            xs_bn = tf.nn.batch_normalization(inputs, mean, var, 
                                              beta, gamma, epsilon)
            
            #xs_norm = (xs - mean) /tf.sqrt(var+eps)
            #xs_bn = (xs_norm * gamma) + beta
            
            return xs_bn    
    
    def test():
        #test할때나 train할때나 beta gamma는 동일함
        #xs_norm = (xs - test_mean) / tf.sqrt(test_var+eps)
        #xs_bn = (xs_norm*gamma) + beta
        #high api사용
        xs_bn = tf.nn.batch_normalization(inputs, test_mean, test_var,
                                          beta, gamma, epsilon)
        
        return xs_bn
        
    xs_bn = tf.cond(is_train, train, test)
    
    return xs_bn
    

In [0]:
def augmentaion(xss, is_train) :
    def train(xs) :
        #xs = tf.image.resize_image_with_crop_or_pad(xs, )
        xs = tf.image.flip_left_right(xs)
        xs = tf.image.random_brightness(xs, 0.8)
        xs = tf.image.random_flip_up_down(xs)
        #xs = tf.image.adjust_contrast(xs, )
        xs = tf.image.random_saturation(xs, 0.5 , 1.8)
        xs = tf.image.per_image_standardization(xs)
        return xs
        
    def test(xs) :
        xs = tf.image.per_image_standardization(xs)
        return xs
    
    
    if is_train == True :
        xs = tf.map_fn(lambda xs : train(xs), xss)
    else :
        xs = tf.map_fn(lambda xs : test(xs), xss)
    return xs

In [0]:
def residual_block(input_data, filters, strides, is_train, block_name) :
    #print('input_shape : ', input_data.shape)
    
    he_init = tf.initializers.he_normal()
    if input_data.shape[-1] != filters :
        projections = tf.layers.Conv2D(filters = filters, kernel_size = 1, 
                                      strides = 1, padding='SAME',
                                      kernel_initializer = he_init)(input_data)
    else :
        projections = input_data
        
    with tf.variable_scope(block_name) :
        con = tf.layers.Conv2D(filters=filters, kernel_size = 3,
                               strides=strides, padding = 'SAME',
                               kernel_initializer = he_init)(projections)
        bn = tf.layers.BatchNormalization()(con, training=is_train)
        #bn = batch_norm_cnn(con, is_train)
        act = tf.nn.relu(bn)
        
        con = tf.layers.Conv2D(filters = filters, kernel_size = 3,
                               strides = strides, padding = 'SAME',
                               kernel_initializer = he_init)\
                               (act)
        bn = tf.layers.BatchNormalization()(con, training=is_train)
        #bn = batch_norm_cnn(con, is_train)

        # Skip-connection layer
        sum_ = tf.add(bn ,input_data)
        
        # activation
        out_data = tf.nn.relu(sum_)
    return out_data
    

In [9]:
graph1 = tf.Graph()
with graph1.as_default() :
    xs = tf.placeholder(dtype = tf.float32, shape=(None, 32, 32, 3))
    ys = tf.placeholder(dtype = tf.int64, shape=(None,))
    is_train = tf.placeholder_with_default(True, (), name='is_train')
    
    xs = augmentaion(xs, is_train)
    
    #first_out = tf.layers.Conv2D(filters = 16, kernel_size = 7, strides = 2,
    #                               padding = 'SAME')
    xa_init = tf.initializers.glorot_normal()
    he_init = tf.initializers.he_normal()

    for i in range(1,5) :
        if i == 1:
            #conv stride=2로 이미지 사이즈를 반으로 줄임
            out = tf.layers.Conv2D(filters = 16, kernel_size = 3, strides = 2,
                                   padding = 'SAME', name= 'conv2_first')(xs)
        else :
            out = residual_block(out, 16, 1, is_train,'conv2_{}'.format(i))
            
    for i in range(1,4) :
        if i == 1:
            out = tf.layers.Conv2D(filters = 32, kernel_size = 3, strides = 2,
                                   padding = 'SAME', name= 'conv3_first')(out)
        else :
            out = residual_block(out, 32, 1, is_train,'conv3_{}'.format(i))
    
    for i in range(1,4) :
        if i == 1:
            out = tf.layers.Conv2D(filters = 64, kernel_size = 3, strides = 2,
                                   padding = 'SAME', name= 'conv4_first')(out)
        else :
            out = residual_block(out, 64, 1, is_train,'conv4_{}'.format(i))
            
    with tf.variable_scope('global_average_pool') :
        # [none, w, h, feature] 에서 w,h에 대해 평균 => image size에 유연함
        gap = tf.reduce_mean(out, axis=(1,2))
        
    logits = tf.layers.Dense(units = 10,
                             activation = None,
                             use_bias = True,
                             kernel_initializer = tf.initializers.glorot_normal())(gap)
    y_pred = tf.nn.softmax(logits)

W0704 11:21:33.447066 140635668719488 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/image_ops_impl.py:1514: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0704 11:21:33.465213 140635668719488 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
 with graph1.as_default() :
    with tf.variable_scope('loss') :
        weight_decay = 1e-4
        sce = tf.losses.sparse_softmax_cross_entropy(labels = ys,
                                                      logits = logits)
        
        weights = graph1.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                       scope='^.*/kernel.*$')
        l2_loss = tf.add_n([tf.nn.l2_loss(weight) 
                            for weight in weights])
        
        loss = sce + weight_decay * l2_loss
    loss = tf.identity(loss, name='loss')  

W0704 11:21:34.938717 140635668719488 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
with graph1.as_default() :
    with tf.variable_scope('metric_acc') :
        pred_val = tf.argmax(y_pred,axis=1)
        acc = tf.reduce_mean(tf.cast(tf.equal(pred_val, ys), tf.float32))

In [0]:
with graph1.as_default() :
    momentum = 0.9
    lr = tf.placeholder(tf.float32, (), name='learning_rate')    
    global_step = tf.train.get_or_create_global_step()
    with tf.variable_scope("optimizer"):
        update_ops = graph1.get_collection(tf.GraphKeys.UPDATE_OPS)
        # update_ops는 moving 계산하는 opration임 -> train_op실행
        # 여기서 train과 test를 control 해줌
        with tf.control_dependencies(update_ops):
            train_op = (tf.train.AdamOptimizer(lr)
                    .minimize(loss, global_step=global_step))

In [13]:
with graph1.as_default() :
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    provider = Dataset(train_x, train_y)
    loss_list = []
    acc_list = []
    loss_val_list = []
    acc_val_list = []
    
    for i in range(10000) :
        data, label = provider.next_batch(100)
        _, loss_, acc_ = sess.run([train_op, loss, acc],
                                              feed_dict = { xs : data,
                                                           ys : label,
                                                          is_train : True,
                                                          lr : 1e-3 })
        
        if i % 100 == 0 :
            loss_val, acc_val = sess.run([loss, acc],
                                              feed_dict = { xs : test_x,
                                                           ys : test_y,
                                                          is_train : False})
            loss_list.append(loss_)
            acc_list.append(acc_)
            loss_val_list.append(loss_val)
            acc_val_list.append(acc_val)
            
            print('step : {}, train_loss : {:5.3}, train_acc : {:5.3}, vali_loss : {:5.3}, vali_acc : {:5.3}'.format(i,loss_, acc_, loss_val, acc_val))
            

step : 0, train_loss :  21.9, train_acc :  0.06, vali_loss :  37.9, vali_acc : 0.118
step : 100, train_loss :  2.13, train_acc :  0.26, vali_loss :  2.39, vali_acc :  0.21
step : 200, train_loss :  2.02, train_acc :  0.34, vali_loss :  2.11, vali_acc : 0.268
step : 300, train_loss :  1.94, train_acc :  0.37, vali_loss :  1.95, vali_acc : 0.318
step : 400, train_loss :  1.71, train_acc :  0.43, vali_loss :  1.91, vali_acc : 0.329
step : 500, train_loss :  1.75, train_acc :  0.41, vali_loss :  1.83, vali_acc : 0.362
step : 600, train_loss :  1.68, train_acc :  0.35, vali_loss :  1.78, vali_acc : 0.371
step : 700, train_loss :  1.54, train_acc :  0.47, vali_loss :  1.72, vali_acc : 0.398
step : 800, train_loss :  1.67, train_acc :  0.45, vali_loss :  1.73, vali_acc : 0.395
step : 900, train_loss :  1.65, train_acc :  0.44, vali_loss :  1.63, vali_acc : 0.427
step : 1000, train_loss :  1.47, train_acc :  0.53, vali_loss :  1.63, vali_acc : 0.423
step : 1100, train_loss :  1.52, train_acc :

In [14]:
with graph1.as_default() :
    for i in range(10000,15000) :
        # Learning rate => 1/10 decay
        data, label = provider.next_batch(100)
        _, loss_, acc_ = sess.run([train_op, loss, acc],
                                              feed_dict = { xs : data,
                                                           ys : label,
                                                          is_train : True,
                                                          lr : 1e-5 })
        
        if i % 1000 == 0 :
            loss_val, acc_val = sess.run([loss, acc],
                                              feed_dict = { xs : test_x,
                                                           ys : test_y,
                                                          is_train : False})
            loss_list.append(loss_)
            acc_list.append(acc_)
            loss_val_list.append(loss_val)
            acc_val_list.append(acc_val)
            
            print('step : {}, train_loss : {:5.3}, train_acc : {:5.3}, vali_loss : {:5.3}, vali_acc : {:5.3}'.format(i,loss_, acc_, loss_val, acc_val))

step : 10000, train_loss : 0.725, train_acc :  0.74, vali_loss :  1.19, vali_acc : 0.606
step : 11000, train_loss : 0.648, train_acc :  0.78, vali_loss :  1.14, vali_acc : 0.622
step : 12000, train_loss : 0.806, train_acc :   0.8, vali_loss :  1.14, vali_acc : 0.624
step : 13000, train_loss : 0.728, train_acc :  0.77, vali_loss :  1.14, vali_acc : 0.626
step : 14000, train_loss : 0.804, train_acc :   0.8, vali_loss :  1.14, vali_acc : 0.625


In [16]:
with graph1.as_default() :
    for i in range(15000,20000) :
        # Learning rate => 1/10 decay
        data, label = provider.next_batch(100)
        _, loss_, acc_ = sess.run([train_op, loss, acc],
                                              feed_dict = { xs : data,
                                                           ys : label,
                                                          is_train : True,
                                                          lr : 1e-5 })
        
        if i % 1000 == 0 :
            loss_val, acc_val = sess.run([loss, acc],
                                              feed_dict = { xs : test_x,
                                                           ys : test_y,
                                                          is_train : False})
            loss_list.append(loss_)
            acc_list.append(acc_)
            loss_val_list.append(loss_val)
            acc_val_list.append(acc_val)
            
            print('step : {}, train_loss : {:5.3}, train_acc : {:5.3}, vali_loss : {:5.3}, vali_acc : {:5.3}'.format(i,loss_, acc_, loss_val, acc_val))

step : 15000, train_loss : 0.856, train_acc :   0.7, vali_loss :  1.14, vali_acc : 0.623
step : 16000, train_loss : 0.735, train_acc :  0.78, vali_loss :  1.15, vali_acc : 0.623
step : 17000, train_loss : 0.668, train_acc :  0.79, vali_loss :  1.15, vali_acc : 0.625
step : 18000, train_loss : 0.532, train_acc :  0.85, vali_loss :  1.15, vali_acc : 0.627
step : 19000, train_loss : 0.774, train_acc :  0.74, vali_loss :  1.15, vali_acc : 0.627
